Two methods:
* Append predictions to modeling dataset, train and predict
* Use predictions as modeling dataset, train and predict

In [1]:
import pandas as pd
import numpy as np
import random

from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold, RandomizedSearchCV, train_test_split

import xgboost as xgb
import matplotlib.pyplot as plt
import os

In [ ]:
train = pd.read_csv("../03-data-creation/train_FE_final.csv")
test = pd.read_csv("../03-data-creation/test_FE_final.csv")

# Drop unwanted column
train = train.drop("Unnamed: 0", axis = 1)
test = test.drop("Unnamed: 0", axis = 1)

# We don't want the ID to be used in the model so preserve
train_id = train['id']
test_id = test['id']

train = train.drop('id', axis = 1)
test = test.drop('id', axis = 1)

# Split X and y
X = train.loc[:, train.columns != "satisfied"]
y = train.satisfied